In [ ]:
### 2. C3D+ucf_model_training_multigpu_training
# Author : @leopauly | cnlp@leeds.ac.uk <br>
# Description : Training the C3D model using UCF 101 action recognition dataset- Testing Phase

print('Started running the program..!',flush=True)
## Imports
from keras.models import Sequential
import random
import numpy as np
from PIL import Image
from os import listdir
from scipy.ndimage import imread
import keras
import tensorflow as tf
from tensorflow.python.client import device_lib
from keras import backend as K
import datetime
import time
import os 
from datetime import timedelta


# Custom scripts
import lscript as lsp
import modelling as md
#from DataSet.DataSet import DataSet
import dataset as dset
import ucf101_dataset as ucf

print('Loaded libraries...!!',flush=True)

In [ ]:
height=112 
width=112 
channel=3
cluster_length=16
nb_classes=101

lr_rate=.001
next_batch_start=0
batch_size=8
batch_size_test=16
total_train_videos=9991
memory_batch_size_train=50
memory_batch_size_test=3329
iterations= 10 # (int(total_train_videos/memory_batch_size_train)) #10001
custom_global_step=0
LOG_DIR='/nobackup/leopauly/logdir'
saved_path='/nobackup/leopauly/logdirk80_2'

print('Finished defining variables..!!',flush=True)

In [ ]:
## Defining placeholders in tf for images and targets
x_image = tf.placeholder(tf.float32, [None, 16,height,width,channel],name='x') 
y_true = tf.placeholder(tf.float32, [None, nb_classes],name='y_true')
y_true_cls = tf.placeholder(tf.int64, [None],name='y_true_cls')

model_keras = md.C3D_ucf101_training_model_tf(summary=False)
out=model_keras(x_image)
y_pred = tf.nn.softmax(out)
y_pred_cls = tf.argmax(out, dimension=1)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print('Miscellenious items finished..!!',flush=True)

In [ ]:
#### Training & testing
def testing(iterations,loops):
    #print(test_images.shape)
    #print(test_labels_cls)
    test_score=0
    for j in range(int(memory_batch_size_test/batch_size_test)-1):
        test_score_= sess.run([accuracy], feed_dict={x_image:test_images[(batch_size_test*j):(batch_size_test*(j+1))],y_true_cls:test_labels_cls[(batch_size_test*j):(batch_size_test*(j+1))],K.learning_phase(): 0 })
        #print('returned value',test_score_)
        print(test_score_)
        test_score=test_score+sum(test_score_)
    print('Test accuracy after iteration:',iterations,',loop:',loops,'is:',test_score/(j+1),flush=True)

In [ ]:
## Loading Testing data
start_time = time.time()
test_images, test_labels_cls, next_batch_start, _ = ucf.read_vid_and_label('./UCF101_data_preparation/test.list',memory_batch_size_test,-1,16,112,normalisation=True)
end_time = time.time()
time_dif = end_time - start_time
print("Time usage for loading training dataset: " + str(timedelta(seconds=int(round(time_dif)))),flush=True)
test_labels=keras.utils.to_categorical(test_labels_cls, num_classes=nb_classes)
print('testing data loaded',flush=True)

In [ ]:
#### Start the session with logging placement.
init_op = tf.global_variables_initializer()
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
sess.run(init_op)

# Restore model weights from previously saved model

#saver = tf.train.import_meta_graph(os.path.join(saved_path,'activity_model_1.ckpt-43.meta'))
saver = tf.train.Saver()
saver.restore(sess, os.path.join(saved_path,'activity_model_1.ckpt-84'))
print("Model restored from file: %s" % saved_path,flush=True)

In [ ]:
start_time = time.time()
testing(0,0)
end_time = time.time()
time_dif = end_time - start_time
print("Time taken for testing: " + str(timedelta(seconds=int(round(time_dif)))),flush=True)

In [ ]:
#sess.close()

In [ ]:
print([n.name for n in tf.get_default_graph().as_graph_def().node])

In [ ]:
for op in tf.get_default_graph().get_operations():
    print(op.name)

In [ ]:
print(tf.contrib.graph_editor.get_tensors(tf.get_default_graph()))

In [ ]:
print(tf.get_default_graph().as_graph_def())

In [ ]:
j=1
batch_size_test=10
vid, test_labels_cls, next_batch_start, _ = ucf.read_vid_and_label('./UCF101_data_preparation/test.list',1,-1,16,112,normalisation=True)
test_labels=keras.utils.to_categorical(test_labels_cls, num_classes=nb_classes)

f_v = sess.graph.get_tensor_by_name('pool5/MaxPool3D:0')
print(f_v)
#f_v_val=sess.run([f_v,accuracy], feed_dict={'conv1_input:0':test_images[(batch_size_test*j):(batch_size_test*(j+1))],y_true:test_labels[(batch_size_test*j):(batch_size_test*(j+1))],x_image:test_images[(batch_size_test*j):(batch_size_test*(j+1))],'y_true_cls:0':test_labels_cls[(batch_size_test*j):(batch_size_test*(j+1))],K.learning_phase(): 0 })
f_v_val=sess.run([f_v], feed_dict={'conv1_input:0':vid,y_true:test_labels,x_image:vid,'y_true_cls:0':test_labels_cls,K.learning_phase(): 0 })
print(np.array(f_v_val).shape)
print(f_v_val)

In [ ]:
f_v_ = sess.graph.get_tensor_by_name('flatten_1/Reshape:0')
print(f_v_)
temp_=2
#f_v_val=sess.run([f_v,accuracy], feed_dict={'conv1_input:0':test_images[(batch_size_test*j):(batch_size_test*(j+1))],y_true:test_labels[(batch_size_test*j):(batch_size_test*(j+1))],x_image:test_images[(batch_size_test*j):(batch_size_test*(j+1))],'y_true_cls:0':test_labels_cls[(batch_size_test*j):(batch_size_test*(j+1))],K.learning_phase(): 0 })
f_v_val_=sess.run([f_v_], feed_dict={'conv1_input:0':vid,x_image:vid,K.learning_phase(): 0 })
print(np.array(f_v_val_).shape)
print(f_v_val_)

temp=np.mean((np.array(f_v_val_))-np.mean(np.array(f_v_val)))

In [ ]:
feature=np.reshape(f_v_val_,(-1))
print(feature.shape)

In [ ]:
print(int(temp))